In [11]:
import pandas as pd
import dateutil.parser

from pandarallel import pandarallel
from datetime import datetime
from tqdm import tqdm

pandarallel.initialize(nb_workers=64)

INFO: Pandarallel will run on 64 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
train_series = pd.read_parquet("../data/train_series.parquet")
train_events = pd.read_csv(
    "../data/train_events.csv",
    parse_dates=["timestamp"],
)

In [4]:
train_events.dropna(subset=["timestamp"], inplace=True)
train_events["step"] = train_events["step"].astype(int)

In [14]:
def parse_timestamp(timestamp):
    return dateutil.parser.isoparse(timestamp)

from concurrent.futures import ProcessPoolExecutor, as_completed

# show progress with tqdm
with ProcessPoolExecutor(max_workers=64) as executor:
    futures = [
        executor.submit(parse_timestamp, timestamp)
        for timestamp in tqdm(train_series["timestamp"].values)
    ]

    timestamps = []
    for future in tqdm(as_completed(futures), total=len(futures)):
        timestamps.append(future.result())

  1%|▏         | 1665831/127946340 [01:58<15:02:44, 2331.41it/s]

: 

In [ ]:
train_series["timestamp"]

In [ ]:
train_series.series_id.value_counts()

In [ ]:
train_events